In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
car_data = pd.read_csv('../input/vehicle-dataset-from-cardekho/car data.csv')

In [ ]:
car_data.head()

In [ ]:
!pip install pandas-profiling

In [ ]:
from pandas_profiling import profile_report

In [ ]:
car_data.profile_report()

In [ ]:
car_data.head()

In [ ]:
print(car_data['Seller_Type'].unique())
print(car_data['Fuel_Type'].unique())
print(car_data['Transmission'].unique())
#print(car_data['Car_Name'].unique())

In [ ]:
car_data.columns

In [ ]:
new_car_data = car_data[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]
new_car_data.head()

* **add new feature**
* get number of year = current year - year

In [ ]:
new_car_data['num_of_year'] = 2021 - new_car_data['Year']
new_car_data.drop(['Year'], axis = 1, inplace= True)


In [ ]:
new_car_data.head()

* **One hot encode the category variables**
* pd.get_dummies()

In [ ]:
new_car_data = pd.get_dummies(new_car_data,drop_first = True)

In [ ]:
new_car_data.corr()

In [ ]:
X = new_car_data.iloc[:,1:]
y = new_car_data.iloc[:,0]


In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
model = ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
import matplotlib.pyplot as plt
print(model.feature_importances_)
feature_imp = pd.Series(model.feature_importances_)
feature_imp.nlargest(8).plot(kind = 'bar')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
X_train.shape

In [ ]:
## hyperparameters tuning

n_estimators = [int(n) for n in np.linspace(start = 100,stop = 1200, num = 12)]
max_features= ['auto','sqrt']
max_depth = [int(n) for n in np.linspace(start = 5, stop = 40, num = 2)]
min_sample_split=[2,5,10,15,100]
min_sample_leaf = [1,2,5,10]
print(n_estimators)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
random = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_sample_split,
    'min_samples_leaf': min_sample_leaf
}
print(random)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model_price = RandomForestRegressor() #n_estimators is decision tree

In [ ]:
random_model = RandomizedSearchCV(estimator = model_price , param_distributions = random, scoring ='neg_mean_squared_error', n_iter = 10, cv= 5)

In [ ]:
random_model.fit(X_train,y_train)

In [ ]:
predictions = random_model.predict(X_test)

In [ ]:
plt.scatter(predictions, y_test)

In [ ]:
from sklearn import metrics
print(metrics.mean_absolute_error(predictions, y_test))

In [ ]:
import pickle
file = open('random_forest.pkl','wb')
pickle.dump(random_model,file)